In [1]:
import cv2
import numpy as np
from PIL import Image
import os
import random
import json
import ast
import csv
from openpyxl import Workbook

## 4. Create

###  4.1 Random player

In [5]:
coordinates_skeld = """{'x_min': 927, 'x_max': 1156, 'y_min': 638, 'y_max': 988}
{'x_min': 915, 'x_max': 1592, 'y_min': 747, 'y_max': 806}
{'x_min': 1408, 'x_max': 1602, 'y_min': 701, 'y_max': 893}
{'x_min': 1467, 'x_max': 1536, 'y_min': 499, 'y_max': 879}
{'x_min': 1562, 'x_max': 1630, 'y_min': 371, 'y_max': 562}
{'x_min': 1569, 'x_max': 1877, 'y_min': 440, 'y_max': 503}
{'x_min': 1743, 'x_max': 1882, 'y_min': 384, 'y_max': 550}
{'x_min': 1318, 'x_max': 1628, 'y_min': 390, 'y_max': 442}
{'x_min': 1475, 'x_max': 1535, 'y_min': 129, 'y_max': 446}
{'x_min': 483, 'x_max': 1532, 'y_min': 218, 'y_max': 265}
{'x_min': 902, 'x_max': 1264, 'y_min': 44, 'y_max': 427}
{'x_min': 1059, 'x_max': 1111, 'y_min': 27, 'y_max': 986}
{'x_min': 1065, 'x_max': 1407, 'y_min': 538, 'y_max': 593}
{'x_min': 1192, 'x_max': 1390, 'y_min': 537, 'y_max': 697}
{'x_min': 670, 'x_max': 853, 'y_min': 305, 'y_max': 515}
{'x_min': 404, 'x_max': 472, 'y_min': 290, 'y_max': 694}
{'x_min': 611, 'x_max': 1161, 'y_min': 829, 'y_max': 892}
{'x_min': 708, 'x_max': 897, 'y_min': 574, 'y_max': 788}
{'x_min': 174, 'x_max': 346, 'y_min': 365, 'y_max': 619}
{'x_min': 182, 'x_max': 638, 'y_min': 469, 'y_max': 534}
{'x_min': 529, 'x_max': 639, 'y_min': 388, 'y_max': 568}
{'x_min': 331, 'x_max': 528, 'y_min': 637, 'y_max': 842}
{'x_min': 326, 'x_max': 519, 'y_min': 153, 'y_max': 343}
{'x_min': 1177, 'x_max': 1386, 'y_min': 832, 'y_max': 990}
{'x_min': 335, 'x_max': 652, 'y_min': 712, 'y_max': 827}
"""

In [9]:
# Usuń białe znaki na początku i końcu
clean_text = coordinates_skeld.strip()

# Podziel na linie
lines = clean_text.splitlines()

# Lista słowników
areas = []

for line in lines:
    line = line.strip()
    if line:
        try:
            area = ast.literal_eval(line)
            areas.append(area)
        except Exception as e:
            print(f"Błąd przy parsowaniu: {line}\n{e}")

# Zapisz do pliku JSON
with open("D:\\Project Sherlock\\spawn_areas.json", "w") as f:
    json.dump(areas, f, indent=4)

print(f"✅ Zapisano {len(areas)} areas do spawn_areas.json")

✅ Zapisano 25 areas do spawn_areas.json


In [6]:
def load_json_file(json_path: str):
    """
    Loads a JSON file and returns its contents.

    Args:
        json_path (str): Path to the JSON file.

    Returns:
        object: Parsed JSON data (e.g., list of dicts).
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"✅ Wczytano {len(data)} areas z {json_path}")
    return data

In [7]:
areas = load_json_file("D:\\Project Sherlock\\spawn_areas.json")

✅ Wczytano 25 areas z D:\Project Sherlock\spawn_areas.json


### 4.3 Put player on map

In [8]:
def overlay_image(background, overlay, x, y, scale_factor):
    """Osadza overlay na background w pozycji (x, y) z obsługą alfa."""
    
    
    
    new_w = int(overlay.shape[1] * scale_factor)
    new_h = int(overlay.shape[0] * scale_factor)

    # Użyj cv2.resize
    overlay = cv2.resize(overlay, (new_w, new_h), interpolation=cv2.INTER_LINEAR)


    bh, bw = background.shape[:2]
    oh, ow = overlay.shape[:2]

    if x + ow > bw or y + oh > bh:
        print("Ostrzeżenie: overlay wykracza poza obraz tła!")
       # print(x+ow, y+oh)
        return background

    # Wyodrębnij kanały
    b, g, r, a = cv2.split(overlay)
    mask = a / 255.0

    for c in range(3):  # RGB
        background[y:y+oh, x:x+ow, c] = (1.0 - mask) * background[y:y+oh, x:x+ow, c] + mask * overlay[:, :, c]

    # Nadpisujemy kanał alfa tła, jeśli potrzebne
    if background.shape[2] == 4:
        background[y:y+oh, x:x+ow, 3] = np.maximum(background[y:y+oh, x:x+ow, 3], a)

    return background

In [9]:
def random_coord_in_area_with_size(area):
    """
    Losuje współrzędne w area tak, żeby cały gracz się zmieścił.
    """

    x = random.randint(area["x_min"], area["x_max"])
    y = random.randint(area["y_min"], area["y_max"])
    return (x, y)

### 4.4 Create screenshots in lopp

In [11]:
def create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc):
    """
    Tworzy obraz postaci z nałożonymi elementami i zwraca go jako numpy array (RGBA).
    
    Args:
        hat_path (str): Ścieżka do pliku czapki.
        mask_path (str): Ścieżka do pliku maski/okularów.
        suit_path (str): Ścieżka do pliku kombinezonu.
        pc_path (str): Ścieżka do bazowej postaci.
    
    Returns:
        np.ndarray: Obraz wynikowy (z kanałem alfa).
    """
    r_pc = random.choice(sprites_pc)
    r_hat = random.choice(sprites_hats)
    r_eye = random.choice(sprites_eyes)
    r_pant = random.choice(sprites_pants)
    
    save_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total"

    # Plik główny (sprite)
    pc_path = "{}\\PC\\{}".format(save_path,  r_pc)
    
    # Dodatkowe elementy
    hat_path = "{}\\Hats\\{}".format(save_path, r_hat)
    mask_path = "{}\\Eyes\\{}".format(save_path,r_eye)
    suit_path = "{}\\Pants\\{}".format(save_path,r_pant)
    # Wczytaj obraz bazowy
    base_img = cv2.imread(pc_path, cv2.IMREAD_UNCHANGED)
    h, w = base_img.shape[:2]
    
    # Nowe wymiary
    new_w = w + 70 + 70
    new_h = h + 160 + 40

    # Tworzymy nowy obraz (przezroczysty)
    new_img = np.zeros((new_h, new_w, 4), dtype=np.uint8)

    # Wklejamy bazowy obraz na nowe płótno
    new_img[160:160+h, 50:50+w] = base_img

    # Wczytaj elementy
    hat = cv2.imread(hat_path, cv2.IMREAD_UNCHANGED)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    suit = cv2.imread(suit_path, cv2.IMREAD_UNCHANGED)

    # Osadź elementy
    extended_img = overlay_image(new_img, hat, 140, 60, scale_factor=1.5)
    extended_img = overlay_image(extended_img, mask, 200, 180, scale_factor=1.3)
    extended_img = overlay_image(extended_img, suit, 98, 355, scale_factor=2.1)

    return extended_img

In [12]:
def spawn_players_on_map_instant(
        sprites_hats, 
        sprites_eyes, 
        sprites_pants,
        sprites_pc,
        map_img,
        spawn_areas,
        output_image_path,
        num_players=1,
        char_width=150,
        char_height=200,
        scale = 4.4609375
                        ):
    """
    Wstawia graczy na mapę w losowych area.
    Zapisuje nowy obraz oraz plik JSON z koordynatami.
    """

    result_img = map_img.copy()

    # Zmień rozmiar postaci
  

    coords_list = []

        
    area = random.choice(spawn_areas)
    for _ in range(num_players):
        coord = random_coord_in_area_with_size(area)
        x, y = coord
        
        x = int(x * scale)
        y = int(y * scale)

        coords_list.append({"x": x, "y": y})
        char_img = create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc)
        char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)
        # Wklej gracza na mapę
        if char_img_resized.shape[2] == 4:
            # Kanał alfa
            alpha_s = char_img_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            for c in range(3):
                result_img[y:y+char_height, x:x+char_width, c] = (
                    alpha_s * char_img_resized[:, :, c] +
                    alpha_l * result_img[y:y+char_height, x:x+char_width, c]
                )
        else:
            result_img[y:y+char_height, x:x+char_width] = char_img_resized

            
    # Koordynaty gracza
    player_coords = {'x': x, 'y': y}

    # Rozdzielczość monitora
    screen_size = (2560, 1440)

    img = result_img
    img_h, img_w = img.shape[:2]
    screen_w, screen_h = screen_size
    center_x, center_y = player_coords['x'], player_coords['y']

    # Oblicz granice cropa
    left = max(center_x - screen_w // 2, 0)
    top = max(center_y - screen_h // 2, 0)
    right = left + screen_w
    bottom = top + screen_h

    if right > img_w:
        right = img_w
        left = max(right - screen_w, 0)

    if bottom > img_h:
        bottom = img_h
        top = max(bottom - screen_h, 0)

    # Przytnij (OpenCV: numpy array)
    cropped = img[top:bottom, left:right]

    # Skalowanie do rozmiaru screena
   # target_size = (1365, 768)  # (width, height)
   # resized = cv2.resize(cropped, target_size, interpolation=cv2.INTER_LANCZOS4)
    resized = zoom_image(cropped)
    # Zapisz obraz

    cv2.imwrite(output_image_path, resized)
  #  print(coords_list)

    return coords_list



In [13]:
def save_coords_to_xlsx(data_list, xlsx_path):
    """
    Zapisuje listę (map_name, list of points) do pliku XLSX.

    Args:
        data_list (list): Lista krotek (map_name, [{'x': x, 'y': y}, ...])
        xlsx_path (str): Ścieżka do pliku XLSX.

    Returns:
        None
    """
    wb = Workbook()
    ws = wb.active
    ws.title = "PlayerCoords"

    # Nagłówki
    ws.append(['map_name', 'point_index', 'x', 'y'])

    for map_name, points in data_list:
        for idx, point in enumerate(points):
            ws.append([map_name, idx, point['x'], point['y']])

    wb.save(xlsx_path)
    print(f"✅ Zapisano dane do {xlsx_path}")


In [14]:
def zoom_image(img, zoom_factor=0.1, target_size=(2560, 1440)):
    """
    Przybliża obraz o zadany procent (zoom_factor=0.1 to 10%).
    Przycinany jest środek obrazu, a następnie skalowany do target_size.
    """
    height, width = img.shape[:2]

    # Oblicz nowe wymiary po cropie (zmniejszenie pola widzenia)
    crop_w = int(width * (1 - zoom_factor))
    crop_h = int(height * (1 - zoom_factor))

    # Współrzędne przycięcia: środek obrazu
    x1 = (width - crop_w) // 2
    y1 = (height - crop_h) // 2
    x2 = x1 + crop_w
    y2 = y1 + crop_h

    # Przytnij i przeskaluj z powrotem do target_size
    cropped = img[y1:y2, x1:x2]
    zoomed = cv2.resize(cropped, target_size, interpolation=cv2.INTER_LANCZOS4)

    return zoomed

### 4.5 Create resized and zoomed screenshots

In [ ]:
areas = load_json_file("D:\\Project Sherlock\\spawn_areas.json")
# Ścieżki
map_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Maps\\The_Skeld_map.jpg"
map_img = cv2.imread(map_path)

sprites_pc = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\PC")[1:]
sprites_eyes = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Eyes")[1:]
sprites_pants = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants")[1:]
sprites_hats = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Hats")[1:]

out_image_coordinates = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\Coordinates_list_per_map.csv"
save_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total"

coordinates_list = []
    
for x in range(750):

    out_image_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\Map_{}.png".format(str(x))
    
    coordinates = spawn_players_on_map_instant(
    
                    map_img=map_img,
                    sprites_hats=sprites_hats, 
                    sprites_eyes=sprites_eyes, 
                    sprites_pants=sprites_pants, 
                    sprites_pc=sprites_pc,

                    spawn_areas=areas,
                    output_image_path=out_image_path,
                    num_players=1    
                                                    )
    
    coordinates_list.append(("Map_{}.png".format(str(x)), coordinates))
   # print("Created Map_{}.png".format(str(x)))

save_coords_to_xlsx(coordinates_list, "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\maps_points.xlsx")

In [45]:
save_coords_to_xlsx(coordinates_list, "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\maps_points.xlsx")

✅ Zapisano dane do D:\Project Sherlock\Grafiki\Grafiki total\Generated_maps\maps_points.xlsx


## 5. Prepare for YALO

In [3]:
import pandas as pd
import os

In [4]:
# PARAMETRY
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
CHAR_WIDTH = 150
CHAR_HEIGHT = 200
MAP_WIDTH = 8565
MAP_HEIGHT = 4794

# Ścieżki
xlsx_path = "E:\\Project Sherlock\\Grafiki\\Grafiki total\\maps_points.xlsx"
output_labels_dir = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\"     # <- katalog wyjściowy na txt
os.makedirs(output_labels_dir, exist_ok=True)

# Wczytaj dane
df = pd.read_excel(xlsx_path)

# Główna pętla
for idx, row in df.iterrows():
    map_name = row["map_name"]
    x = row["x"]
    y = row["y"]

    # Oblicz pozycję viewportu (screen) na mapie
    left = max(x - SCREEN_WIDTH // 2, 0)
    top = max(y - SCREEN_HEIGHT // 2, 0)
    right = min(left + SCREEN_WIDTH, MAP_WIDTH)
    bottom = min(top + SCREEN_HEIGHT, MAP_HEIGHT)

    # Korekta: jeżeli prawa lub dolna krawędź wypadnie poza mapę
    if right - left < SCREEN_WIDTH:
        left = max(MAP_WIDTH - SCREEN_WIDTH, 0)
    if bottom - top < SCREEN_HEIGHT:
        top = max(MAP_HEIGHT - SCREEN_HEIGHT, 0)

    # Pozycja gracza względem screena
    x_on_screen = x - left
    y_on_screen = y - top

    # Pozycja bboxa na screenie (YOLO format)
    bbox_center_x = x_on_screen + CHAR_WIDTH // 2
    bbox_center_y = y_on_screen + CHAR_HEIGHT // 2

    bbox_width = CHAR_WIDTH
    bbox_height = CHAR_HEIGHT

    # YOLO format: wszystko znormalizowane do [0–1]
    x_norm = bbox_center_x / SCREEN_WIDTH
    y_norm = bbox_center_y / SCREEN_HEIGHT
    w_norm = bbox_width / SCREEN_WIDTH
    h_norm = bbox_height / SCREEN_HEIGHT

    # Sprawdź, czy bbox mieści się na screenie
    if 0 <= x_norm <= 1 and 0 <= y_norm <= 1:
        yolo_line = f"0 {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}\n"
    else:
        # Gracz poza ekranem (teoretycznie niemożliwe, ale zabezpieczamy się)
        yolo_line = ""

    # Zapisz etykietę
    label_file = os.path.join(output_labels_dir, map_name.replace(".png", ".txt"))
    with open(label_file, "w") as f:
        if yolo_line:
            f.write(yolo_line)


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Project Sherlock\\Grafiki\\Grafiki total\\maps_points.xlsx'

### 5.2 Split datatest 

In [110]:
all_files  = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels")

In [112]:
files_txt =[]
for x in all_files:
    if ".txt" in x:
        files_txt.append(x)

In [113]:
files_txt

['Map_0.txt',
 'Map_1.txt',
 'Map_10.txt',
 'Map_100.txt',
 'Map_101.txt',
 'Map_102.txt',
 'Map_103.txt',
 'Map_104.txt',
 'Map_105.txt',
 'Map_106.txt',
 'Map_107.txt',
 'Map_108.txt',
 'Map_109.txt',
 'Map_11.txt',
 'Map_110.txt',
 'Map_111.txt',
 'Map_112.txt',
 'Map_113.txt',
 'Map_114.txt',
 'Map_115.txt',
 'Map_116.txt',
 'Map_117.txt',
 'Map_118.txt',
 'Map_119.txt',
 'Map_12.txt',
 'Map_120.txt',
 'Map_121.txt',
 'Map_122.txt',
 'Map_123.txt',
 'Map_124.txt',
 'Map_125.txt',
 'Map_126.txt',
 'Map_127.txt',
 'Map_128.txt',
 'Map_129.txt',
 'Map_13.txt',
 'Map_130.txt',
 'Map_131.txt',
 'Map_132.txt',
 'Map_133.txt',
 'Map_134.txt',
 'Map_135.txt',
 'Map_136.txt',
 'Map_137.txt',
 'Map_138.txt',
 'Map_139.txt',
 'Map_14.txt',
 'Map_140.txt',
 'Map_141.txt',
 'Map_142.txt',
 'Map_143.txt',
 'Map_144.txt',
 'Map_145.txt',
 'Map_146.txt',
 'Map_147.txt',
 'Map_148.txt',
 'Map_149.txt',
 'Map_15.txt',
 'Map_150.txt',
 'Map_151.txt',
 'Map_152.txt',
 'Map_153.txt',
 'Map_154.txt',
 '

In [116]:
main_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels"
for x in range(len(files_txt)):
    os.rename("{}\\{}".format(main_path, files_txt[x]),
              "{}\\{}".format(main_path, "Label_image_{}.txt".format(str(x))))

In [123]:
main_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset"
files_img =[]
all_files  = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset")
for x in all_files:
    if "." in x:
        files_img.append(x)

In [124]:
files_img

['Image_0.png',
 'Image_1.png',
 'Image_10.png',
 'Image_100.png',
 'Image_101.png',
 'Image_102.png',
 'Image_103.png',
 'Image_104.png',
 'Image_105.png',
 'Image_106.png',
 'Image_107.png',
 'Image_108.png',
 'Image_109.png',
 'Image_11.png',
 'Image_110.png',
 'Image_111.png',
 'Image_112.png',
 'Image_113.png',
 'Image_114.png',
 'Image_115.png',
 'Image_116.png',
 'Image_117.png',
 'Image_118.png',
 'Image_119.png',
 'Image_12.png',
 'Image_120.png',
 'Image_121.png',
 'Image_122.png',
 'Image_123.png',
 'Image_124.png',
 'Image_125.png',
 'Image_126.png',
 'Image_127.png',
 'Image_128.png',
 'Image_129.png',
 'Image_13.png',
 'Image_130.png',
 'Image_131.png',
 'Image_132.png',
 'Image_133.png',
 'Image_134.png',
 'Image_135.png',
 'Image_136.png',
 'Image_137.png',
 'Image_138.png',
 'Image_139.png',
 'Image_14.png',
 'Image_140.png',
 'Image_141.png',
 'Image_142.png',
 'Image_143.png',
 'Image_144.png',
 'Image_145.png',
 'Image_146.png',
 'Image_147.png',
 'Image_148.png',
 

In [126]:
main_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset"
for x in range(len(files_img)):
    os.rename("{}\\{}".format(main_path, files_img[x]),
              "{}\\{}".format(main_path, "New_Image_{}.png".format(str(x))))

In [144]:
files_image = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\images")
files_labels = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels")

In [129]:
import random


files = list(range(0,1500))  # żeby nie modyfikować oryginału
random.shuffle(files)

# Określ proporcje
total = len(files)
test_split = int(0.2 * total)
val_split = int(0.2 * total)  # możesz zmienić np. na 0.1
train_split = total - test_split - val_split

# Podział bez zwracania
test = files[:test_split]
val = files[test_split:test_split+val_split]
train = files[test_split+val_split:]

In [130]:
total

1500

In [134]:
len(test)

300

In [135]:
len(train)

900

In [136]:
len(val)

300

### 5.3 Distribute the files to specific folders

#### 5.3.1 Images

In [ ]:
files_image = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\images")
files_labels = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels")

In [146]:
for x in test:
    os.rename("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\images\\{}".format("New_Image_{}.png".format(str(x))),
             "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\images\\test\\{}".format("New_Image_{}.png".format(str(x))))

In [147]:
for x in val:
    os.rename("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\images\\{}".format("New_Image_{}.png".format(str(x))),
             "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\images\\val\\{}".format("New_Image_{}.png".format(str(x))))

In [148]:
for x in train:
    os.rename("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\images\\{}".format("New_Image_{}.png".format(str(x))),
             "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\images\\train\\{}".format("New_Image_{}.png".format(str(x))))

#### 5.3.2 Labels

In [164]:
for x in os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels"):
    if "." in x:
        os.rename("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\{}".format(x),
             "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\{}".format("New_Image_{}.txt".format(x.split(".")[0].split("_")[-1])))

In [165]:
for x in test:
    os.rename("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\{}".format("New_Image_{}.txt".format(str(x))),
             "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\test\\{}".format("New_Image_{}.txt".format(str(x))))

In [166]:
for x in train:
    os.rename("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\{}".format("New_Image_{}.txt".format(str(x))),
             "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\train\\{}".format("New_Image_{}.txt".format(str(x))))

In [167]:
for x in val:
    os.rename("D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\{}".format("New_Image_{}.txt".format(str(x))),
             "D:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\labels\\val\\{}".format("New_Image_{}.txt".format(str(x))))

### 5.4 Begin training

In [6]:
from ultralytics import YOLO
from ultralytics import settings

WARNING Ultralytics setting 'datasets_dir: D:\Project Sherlock\Grafiki\Grafiki total\dataset' must be different than 'runs_dir: D:\Project Sherlock\Grafiki\Grafiki total\dataset'. Please change one to avoid possible issues during training. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Jarek\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
# Update a setting
settings.update({"runs_dir": "E:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset"})
settings.update({"datasets_dir": "E:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset"})
settings.update({"weights_dir": "E:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset"})

In [14]:
model = YOLO("E:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\yolov8n.pt")  

In [15]:
model.train(
    data='E:\\Project Sherlock\\Grafiki\\Grafiki total\\dataset\\data.yaml',
    epochs=1,
    imgsz=640,
    device='cpu'
)

Ultralytics 8.3.168  Python-3.10.18 torch-2.7.1+cpu CPU (AMD Ryzen 5 5600X 6-Core Processor)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:\Project Sherlock\Grafiki\Grafiki total\dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=E:\Project Sherlock\Grafiki\Grafiki total\dataset\yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer

train: Scanning E:\Project Sherlock\Grafiki\Grafiki total\dataset\labels\train... 900 images, 462 backgrounds, 0 corrup

train: New cache created: E:\Project Sherlock\Grafiki\Grafiki total\dataset\labels\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 3645.5286.7 MB/s, size: 2413.1 KB)



D:\anaconda\envs\yalo_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning E:\Project Sherlock\Grafiki\Grafiki total\dataset\labels\test... 300 images, 146 backgrounds, 0 corrupt: 

val: New cache created: E:\Project Sherlock\Grafiki\Grafiki total\dataset\labels\test.cache


...

        1/1         0G      1.455      3.955      1.118          3        640: 100%|██████████| 57/57 [03:25<00:00,  3.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:22


                   all        300        154    0.00133      0.773    0.00396    0.00254

1 epochs completed in 0.064 hours.
Optimizer stripped from D:\Project Sherlock\Grafiki\Grafiki total\dataset\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from D:\Project Sherlock\Grafiki\Grafiki total\dataset\detect\train4\weights\best.pt, 6.2MB

Validating D:\Project Sherlock\Grafiki\Grafiki total\dataset\detect\train4\weights\best.pt...
Ultralytics 8.3.168  Python-3.10.18 torch-2.7.1+cpu CPU (AMD Ryzen 5 5600X 6-Core Processor)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:20


                   all        300        154    0.00133      0.773    0.00395    0.00254
Speed: 0.8ms preprocess, 28.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to D:\Project Sherlock\Grafiki\Grafiki total\dataset\detect\train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000239A995E410>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
results_dict: {
  'metrics/precision(B)': 0.0013,        # precyzja
  'metrics/recall(B)': 0.7727,           # czułość (recall)
  'metrics/mAP50(B)': 0.00395,           # mAP@0.5 (średnia precyzja przy IoU=0.5)
  'metrics/mAP50-95(B)': 0.00253,        # mAP@[0.5:0.95] (uśredniona dokładność)
  'fitness': 0.0027                      # wartość do optymalizacji modelu
}